In [1]:
import pandas as pd
import math
import os
import networkx as nx
import numpy as np
import scipy.sparse as sp
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from deep_translator import GoogleTranslator

In [2]:
input_path="./data/ranked/"
output_path="./data/summaries/"
if os.path.exists(output_path) == False:
    os.mkdir(output_path)

def clean_text(row):
    text = []
    [text.extend(i.strip().split('।')) for i in row]
    text = [i.strip() for i in text]
    text = list(filter(None, text))
    return text



def get_sentences():
    train = pd.read_csv(f'{input_path}train.csv')
    test = pd.read_csv(f'{input_path}test.csv')
    val = pd.read_csv(f'{input_path}val.csv')
    #Comment
    train = train.head(1)
    test = test.head(1)
    val = val.head(1)
    print(train.shape, test.shape, val.shape)
    #Comment

    train['segments'] = train['segments'].apply(eval)
    test['segments'] = test['segments'].apply(eval)
    val['segments'] = val['segments'].apply(eval)

    train_sentences = train['segments'].apply(lambda x: clean_text(x['facts-and-arguments']))
    test_sentences = test['segments'].apply(lambda x: clean_text(x['facts-and-arguments']))
    val_sentences = val['segments'].apply(lambda x: clean_text(x['facts-and-arguments']))

    return train,test,val,train_sentences, val_sentences, test_sentences

train,test,val,train_sentences, val_sentences, test_sentences = get_sentences()
print(test_sentences[0])

(1, 6) (1, 6) (1, 6)
['जमानत प्रार्थनापत्र के समर्थन में अभियुक्त के पैरोकार शरीफ उर्फशफीक ने शपथ पत्र प्रस्तुत किया है', 'अभियोजन कथानक संक्षेप में इस प्रकार है कि वादी मुकदमा लक्ष्मीनरायन ने थाना शाहगंज <नाम> दिनाक 09.08.2020 को इस आशय की तहरीर दी किउसका स्कूटर मेस्ट्रो रंग स्काई ब्लू नम्बर यूपी.80 डी.डी.2730 दिनांक 22.07.2020 कोरात्रि 9.30 बजे उसके निवास <नाम> ताला लगा खड़ी थी, तभी समय करीब 10 बजेउसने <नाम> तो उसके उक्त स्कूटर को अज्ञात चोर चोरी करके ले गया है', 'उसकेपड़ौस में लगे सी.सी.टी.वी.कैमरे में देखने <नाम> तीन अज्ञात चोर 9.54 पी.एम. <नाम> स्कूटरमें चाबी लगाकर चोरी करके स्कूटर ले जाते दिखाई <नाम> रहे थे', 'फुटेज की क्लिपउसके पास मौजूद है', 'इस घटना की जानकारी उसने तत्काल 112 नं0 <नाम> <नाम> दीशी', 'काफी खोजबीन करने <नाम> उसकी उक्त स्कूटी नहीं <नाम> है', 'वादी मुकदमा की उक्त तहरीर के आधार <नाम> थाना शाहगंज <नाम> अज्ञातके विरूद्ध मु0अ0सं0 3612020 अंतर्गत <नाम> 379 भा0द0सं0 पंजीकृत हुआ', '<नाम> मेंप्रकरण में अभियुक्त के पकड़े जाने <नाम> <नाम> 411 भा0द0सं0 की बढोत्तरी की गयी है',

In [3]:
def textrank(document):
    sentence_tokenizer = PunktSentenceTokenizer()
    sentences = sentence_tokenizer.tokenize(document)

    bow_matrix = CountVectorizer().fit_transform(sentences)
    normalized = TfidfTransformer().fit_transform(bow_matrix)

    similarity_graph = normalized * normalized.T
    sparse_matrix = sp.csr_matrix(similarity_graph)
    nx_graph = nx.Graph(sparse_matrix)
    scores = nx.pagerank(nx_graph)
    sentence_array = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    
    sentence_array = np.asarray(sentence_array)
    
    fmax = float(sentence_array[0][0])
    fmin = float(sentence_array[len(sentence_array) - 1][0])
    
    temp_array = []
    for i in range(0, len(sentence_array)):
        if fmax - fmin == 0:
        	temp_array.append(0)
        else:
        	temp_array.append((float(sentence_array[i][0]) - fmin) / (fmax - fmin))

    threshold = (sum(temp_array) / len(temp_array)) + 0.2
    
    sentence_list = []

    for i in range(0, len(temp_array)):
        if temp_array[i] > threshold:
            sentence_list.append(sentence_array[i][1])

    seq_list = []
    for sentence in sentences:
    	if sentence in sentence_list:
    		seq_list.append(sentence)
    
    return seq_list

In [4]:
def split_long_sentence(sentence, max_length=3000):
    parts = []
    for i in range(0, len(sentence), max_length):
        part = sentence[i:i + max_length]
        parts.append(part)

    return parts

In [5]:
def summarize(sentences):
    result = ''
    for sentence_array in sentences:
        sentence = ''
        for i in range(len(sentence_array)):
            sentence = sentence + sentence_array[i]
        if len(sentence)>2000:
            small_sentences = split_long_sentence(sentence, max_length=2000)
            engText = ''
            for i in range(len(small_sentences)):
                current_translate = GoogleTranslator(source='hi', target='en').translate(small_sentences[i])
                engText = engText + current_translate
        else:
            engText = GoogleTranslator(source='hi', target='en').translate(sentence)
        result=result +engText
    return textrank(result)

train_engSum = []
val_engSum = []
test_engSum = []


for i in range(len(train_sentences)):
    train_engSum.append(summarize(train_sentences[i]))

for i in range(len(val_sentences)):
    val_engSum.append(summarize(val_sentences[i]))

for i in range(len(test_sentences)):
    test_engSum.append(summarize(test_sentences[i]))

train['engSum'] = train_engSum
val['engSum'] = val_engSum
test['engSum'] = test_engSum

train.to_csv(f'{output_path}train.csv', index=False)
val.to_csv(f'{output_path}val.csv', index=False)
test.to_csv(f'{output_path}test.csv', index=False)
